In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

# 读取数据，第一行为文本，第二行为标签
def load_data(filename):
    texts = []
    labels = []
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for i in range(0, len(lines), 2):  # 每两行一起处理
            text = lines[i].strip()
            label = lines[i+1].strip()
            try:
                labels.append(int(label))  # 尝试将标签转换为整数
                texts.append(text)
            except ValueError:
                print(f"Skipping invalid label: {label}")
    return texts, labels

# 文本分词
def tokenize(text):
    return re.findall(r'\w+', text.lower())

# 构建词汇表
def build_vocab(texts):
    vocab = {}
    idx = 0
    for text in texts:
        tokens = tokenize(text)
        for token in tokens:
            if token not in vocab:
                vocab[token] = idx
                idx += 1
    return vocab

# 将文本转换为特征向量
def text_to_features(texts, vocab):
    features = []
    for text in texts:
        tokens = tokenize(text)
        token_ids = [vocab.get(token, -1) for token in tokens if token in vocab]
        features.append(token_ids)
    return features

# 填充序列
def pad_sequences(sequences, maxlen):
    return torch.tensor([seq + [0] * (maxlen - len(seq)) for seq in sequences])

# 加载训练和测试数据
train_texts, train_labels = load_data('train.txt')
test_texts, test_labels = load_data('test.txt')

# 构建词汇表
vocab = build_vocab(train_texts)

# 将文本转换为特征
train_features = text_to_features(train_texts, vocab)
test_features = text_to_features(test_texts, vocab)

# 标签编码
le = LabelEncoder()
train_labels = torch.tensor(le.fit_transform(train_labels))
test_labels = torch.tensor(le.transform(test_labels))

# 确定最大序列长度并填充
maxlen = max(max(len(seq) for seq in train_features), max(len(seq) for seq in test_features))
train_features = pad_sequences(train_features, maxlen)
test_features = pad_sequences(test_features, maxlen)

In [20]:
class GGNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(GGNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)  # 词嵌入
        output, hidden = self.gru(embedded)  # GRU更新节点表示
        hidden = hidden.squeeze(0)
        return self.fc(hidden)


In [21]:
# 训练模型
def train_model(model, train_features, train_labels, epochs=20, batch_size=32):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(0, len(train_features), batch_size):
            batch_features = train_features[i:i+batch_size]
            batch_labels = train_labels[i:i+batch_size]
            
            optimizer.zero_grad()
            output = model(batch_features)
            loss = loss_fn(output, batch_labels)
            loss.backward()
            optimizer.step()
            
            # 计算损失
            total_loss += loss.item()
            
            # 计算每个批次的准确率和F1
            preds = torch.argmax(output, dim=1).cpu().numpy()
            labels = batch_labels.cpu().numpy()
            batch_acc = accuracy_score(labels, preds)
            batch_f1 = f1_score(labels, preds, average='macro')
            print(f'Epoch {epoch+1}, Batch {i//batch_size+1}: ACC = {batch_acc:.4f}, F1 = {batch_f1:.4f}')
        
        # 输出每个epoch的平均损失
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_features):.4f}')


# 测试模型
def test_model(model, test_features, test_labels, batch_size=32):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for i in range(0, len(test_features), batch_size):
            batch_features = test_features[i:i+batch_size]
            batch_labels = test_labels[i:i+batch_size]
            
            output = model(batch_features)
            preds = torch.argmax(output, dim=1).cpu().numpy()
            labels = batch_labels.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)
            
            # 打印每个批次的准确率和F1
            batch_acc = accuracy_score(labels, preds)
            batch_f1 = f1_score(labels, preds, average='macro')
            print(f'Batch {i//batch_size+1}: ACC = {batch_acc:.4f}, F1 = {batch_f1:.4f}')
    
    # 计算整体测试集的准确率和F1
    test_acc = accuracy_score(all_labels, all_preds)
    test_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f'Test Accuracy: {test_acc:.4f}, F1-score: {test_f1:.4f}')


In [22]:
# 模型参数
vocab_size = len(vocab)
embed_size = 64
hidden_size = 128
num_classes = len(set(train_labels.tolist()))

# 初始化模型
model = GGNN(vocab_size, embed_size, hidden_size, num_classes)

# 损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
train_model(model, train_features, train_labels, epochs=20, batch_size=32)

# 测试模型
test_model(model, test_features, test_labels, batch_size=32)


Epoch 1, Batch 1: ACC = 0.6875, F1 = 0.4074
Epoch 1, Batch 2: ACC = 0.5312, F1 = 0.3469
Epoch 1, Batch 3: ACC = 0.6562, F1 = 0.3962
Epoch 1, Batch 4: ACC = 0.5000, F1 = 0.3333
Epoch 1, Batch 5: ACC = 0.5312, F1 = 0.3469
Epoch 1, Batch 6: ACC = 0.4688, F1 = 0.3191
Epoch 1, Batch 7: ACC = 0.3125, F1 = 0.2381
Epoch 1, Batch 8: ACC = 0.5938, F1 = 0.3725
Epoch 1, Batch 9: ACC = 0.3125, F1 = 0.2381
Epoch 1, Batch 10: ACC = 0.5625, F1 = 0.3600
Epoch 1, Batch 11: ACC = 0.5000, F1 = 0.3333
Epoch 1, Batch 12: ACC = 0.5938, F1 = 0.4793
Epoch 1, Batch 13: ACC = 0.6250, F1 = 0.3846
Epoch 1, Batch 14: ACC = 0.5000, F1 = 0.3333
Epoch 1, Batch 15: ACC = 0.4062, F1 = 0.2889
Epoch 1, Batch 16: ACC = 0.5625, F1 = 0.3600
Epoch 1, Batch 17: ACC = 0.6562, F1 = 0.3962
Epoch 1, Batch 18: ACC = 0.5000, F1 = 0.3333
Epoch 1, Batch 19: ACC = 0.5000, F1 = 0.3333
Epoch 1, Batch 20: ACC = 0.7500, F1 = 0.4286
Epoch 1, Batch 21: ACC = 0.5938, F1 = 0.3725
Epoch 1, Batch 22: ACC = 0.6250, F1 = 0.3846
Epoch 1, Batch 23: 